In [27]:
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import pickle
from tqdm import tqdm
import warnings
import time
import torch

In [3]:
with open('../DeepTimeGeo/trained_models/rb_0711_mask_0/data/training_set.pkl', 'rb') as f:
    loc_np, time_x, poi_x, uid, loc_y, loc_y_exploration, is_commuter, rank_dist, selected_home_rank, selected_work_rank = pickle.load(f)
key_table = pd.read_csv('../DeepTimeGeo/data/coral_gables/0708_r45_100/coral_gables_dict.csv')[['id', 'lon_x', 'lat_x', 'rank']]

In [4]:
loc_np = np.array(loc_np)
uid_np = np.array(uid)

In [5]:
from tqdm import tqdm
loc_np_filled = loc_np.copy()
for i in tqdm(range(loc_np.shape[0])):
    init = loc_np[i,:][loc_np[i,:] != 0][0]
    for j in range(loc_np.shape[1]):
        if loc_np[i,j] == 0:
            loc_np_filled[i,j] = init
        else:
            init = loc_np[i,j]
            loc_np_filled[i,j] = init


  0%|          | 0/34147 [00:00<?, ?it/s]

100%|██████████| 34147/34147 [00:03<00:00, 9752.41it/s] 


In [6]:
arr_id = np.repeat(uid_np, loc_np_filled.shape[1])
arr_loc = loc_np_filled.flatten()

In [7]:
filled = pd.DataFrame({'id': arr_id, 'rank': arr_loc}, dtype='int64')
merged = pd.merge(filled, key_table, on=['id', 'rank'], how='left')

In [8]:
loc_key = merged[['lon_x', 'lat_x']].drop_duplicates().reset_index(drop=True).reset_index()
final = pd.merge(merged, loc_key, on=['lon_x', 'lat_x'], how='left')

In [23]:
real_data = final['index'].to_numpy().reshape(loc_np_filled.shape)

fake_data = real_data.copy()
indices = np.random.choice(real_data.size, int(real_data.size*0.2), replace=False)
random_replacement = np.random.randint(0, loc_key.shape[0], size=indices.size)
fake_data_flat = fake_data.flatten()
fake_data_flat[indices] = random_replacement
fake_data = fake_data_flat.reshape(fake_data.shape)

real_data = torch.tensor(real_data, dtype=torch.long)
fake_data = torch.tensor(fake_data, dtype=torch.long)

data = torch.cat([real_data, fake_data], dim=0)

In [26]:
torch.save(data, 'SeqGAN_input.pt')

# Evaluate Outputs

In [43]:
import generator
gen = generator.Generator(32, 32, 81251, 24*14, gpu=True)
gen.load_state_dict(torch.load('gen_adv_trained.trc'))
gen.to(torch.device('cuda:1'))
generated_sample = gen.sample(10000)

In [44]:
np.save('SeqGAN_output.npy', generated_sample.cpu().numpy())